In [ ]:
!nvidia-smi

In [ ]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [2]:

import pandas as pd
import tensorflow as tf

import glob
import re

In [ ]:
%pip install fastparquet

In [3]:
base_path = '/home/workspace/geometry-classifier/data/'



In [14]:
!du -h /home/workspace/geometry-classifier/data/tfrecord

16M	/home/workspace/geometry-classifier/data/tfrecord


In [12]:
import pandas as pd
import tensorflow as tf
import glob
import os

def serialize_example(name, image_data):
    """Serialize data into TFRecord format."""
    feature = {
        'name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[name])),
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_data]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def parquet_to_tfrecord(parquet_file, tfrecord_file):
    """Convert a single Parquet file to a TFRecord file."""
    df = pd.read_parquet(parquet_file).drop(columns=["id"])
    print(df.shape)
    
    with tf.io.TFRecordWriter(tfrecord_file, options='GZIP') as writer:
        for _, row in df.iterrows():
            name = row['name'].astype('uint8').tobytes()  # Convert uint8 name to bytes
            image_data = row.drop('name').to_numpy().astype('uint8').tobytes()  # Flatten image data and convert to bytes
            example = serialize_example(name, image_data)
            writer.write(example)

# Directory with parquet files
parquet_dir = base_path+"photos_v3_parquet"

parquet_files = glob.glob(os.path.join(parquet_dir, "*.parquet"))

i = 0
for parquet_file in parquet_files:
    if i == 10: break
    i+=1
    tfrecord_file = f"{base_path}tfrecord/data{i}.tfrecord.gz"
    parquet_to_tfrecord(parquet_file, tfrecord_file)
    print(f"Converted {parquet_file} to {tfrecord_file}")


(1024, 50177)
Converted /home/workspace/geometry-classifier/data/photos_v3_parquet/photos_649.parquet to /home/workspace/geometry-classifier/data/tfrecord/data1.tfrecord.gz
(1024, 50177)
Converted /home/workspace/geometry-classifier/data/photos_v3_parquet/photos_432.parquet to /home/workspace/geometry-classifier/data/tfrecord/data2.tfrecord.gz
(1024, 50177)
Converted /home/workspace/geometry-classifier/data/photos_v3_parquet/photos_82.parquet to /home/workspace/geometry-classifier/data/tfrecord/data3.tfrecord.gz
(1024, 50177)
Converted /home/workspace/geometry-classifier/data/photos_v3_parquet/photos_592.parquet to /home/workspace/geometry-classifier/data/tfrecord/data4.tfrecord.gz
(1024, 50177)
Converted /home/workspace/geometry-classifier/data/photos_v3_parquet/photos_838.parquet to /home/workspace/geometry-classifier/data/tfrecord/data5.tfrecord.gz
(1024, 50177)
Converted /home/workspace/geometry-classifier/data/photos_v3_parquet/photos_272.parquet to /home/workspace/geometry-classi